# Construção da ABT
_ABT - Analytical Base Table_

---

## Sumário

1. **Importação da bibliotecas**
2. **Criação e iniciação de uma sessão PySpark**
3. **Criação dos datasets a partir da leitura dos arquivos \*.csv e \*.json**
    - 3.1. Arquivos CSV
    - 3.2. Arquivos JSON
4. **Análise dos dados**
5. **Construção da ABT**
    - 5.1. Análise de dados da ABT
    - 5.2. Análise do target da ABT
    - 5.3. Gerando uma amostra representativa da ABT
6. **Salvando a ABT em formato parquet**

<br>

---

<br>

## 1. Importação de bibliotecas

In [49]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, col, expr, count, lit
import os
import json
import csv

## 2. Criação e iniciação de uma sessão PySpark

In [50]:
appName = 'ABT - Transações de clientes'

# O objeto SparkSession configurado com configurações específicas
spark = SparkSession.builder \
    .appName(appName) \
    .config('spark.driver.memory', '8g') \
    .config('spark.executor.memory', '8g') \
    .config('spark.master', 'local[*]') \
    .getOrCreate()
    
spark.sparkContext.setLogLevel('ERROR')

spark

## 3. Criação dos datasets a partir da leitura dos arquivos *.csv e *.json

In [51]:
# Caminho dos arquivos *.csv e *.json
caminho = f'dados/'

### 3.1. Arquivos CSV

- cards_data.csv

In [52]:
# Dados de cartão de crédito
df_cards = spark.read.csv(caminho + 'cards_data.csv', header=True, inferSchema=True)

# Exibir o esquema do dataframe
df_cards.printSchema()

root
 |-- id: integer (nullable = true)
 |-- client_id: integer (nullable = true)
 |-- card_brand: string (nullable = true)
 |-- card_type: string (nullable = true)
 |-- card_number: long (nullable = true)
 |-- expires: string (nullable = true)
 |-- cvv: integer (nullable = true)
 |-- has_chip: string (nullable = true)
 |-- num_cards_issued: integer (nullable = true)
 |-- credit_limit: string (nullable = true)
 |-- acct_open_date: string (nullable = true)
 |-- year_pin_last_changed: integer (nullable = true)
 |-- card_on_dark_web: string (nullable = true)



- transactions_data.csv

In [53]:
# Dados de transações
df_transactions = spark.read.csv(caminho + 'transactions_data.csv', header=True, inferSchema=True)

# Exibir o esquema do dataframe
df_transactions.printSchema()

root
 |-- id: integer (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- client_id: integer (nullable = true)
 |-- card_id: integer (nullable = true)
 |-- amount: string (nullable = true)
 |-- use_chip: string (nullable = true)
 |-- merchant_id: integer (nullable = true)
 |-- merchant_city: string (nullable = true)
 |-- merchant_state: string (nullable = true)
 |-- zip: double (nullable = true)
 |-- mcc: integer (nullable = true)
 |-- errors: string (nullable = true)



- users_data.csv

In [54]:
# Dados de clientes
df_clients = spark.read.csv(caminho + 'users_data.csv', header=True, inferSchema=True)

# Exibir o esquema do dataframe
df_clients.printSchema()

root
 |-- id: integer (nullable = true)
 |-- current_age: integer (nullable = true)
 |-- retirement_age: integer (nullable = true)
 |-- birth_year: integer (nullable = true)
 |-- birth_month: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- address: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- per_capita_income: string (nullable = true)
 |-- yearly_income: string (nullable = true)
 |-- total_debt: string (nullable = true)
 |-- credit_score: integer (nullable = true)
 |-- num_credit_cards: integer (nullable = true)



### 3.2. Arquivos JSON

In [55]:
def process_json_to_csv(json_path, csv_path, list_columns, one_column=None):
    '''
    Processa um arquivo JSON e converte em CSV.

    :param json_path: path + filename.json
        Caminho do arquivo JSON de entrada.
    :param csv_path: path + filename.csv
        Caminho do arquivo CSV de saída.
    :param list_columns: list
        Lista com os nomes das colunas do CSV.
    :param one_column: string (default=None)
        Nome da chave no JSON para acessar dados aninhados (opcional).
    '''
    # Carregar o arquivo JSON
    with open(json_path, 'r', encoding='utf-8') as json_file:
        data = json.load(json_file)
        
    if not data:
        print('Arquivo JSON está vazio.')
        return
    
    # Processar dados aninhados, se necessário
    if one_column:
        data = data.get(one_column, {})
        if not data:
            print(f'A chave \'{one_column}\' não foi encontrada ou está vazia no JSON.')
            return
        print(f'Dados extraídos da chave \'{one_column}\'.')
    else:
        print('Processando JSON sem dados aninhados.')

    # Escrever o CSV
    with open(csv_path, 'w', encoding='utf-8', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(list_columns)  # Escrevendo o cabeçalho
        
        for key, value in data.items():
            writer.writerow([key, value])  # Escrevendo as linhas
    
    print(f'Arquivo CSV gerado com sucesso em {csv_path}.')

- mcc_codes.json

In [56]:
# Processar mcc_codes.json
json_path = caminho + 'mcc_codes.json'
csv_path = caminho + 'mcc_codes.csv'
list_columns = ['code', 'description']
process_json_to_csv(json_path, csv_path, list_columns)

Processando JSON sem dados aninhados.
Arquivo CSV gerado com sucesso em dados/mcc_codes.csv.


In [57]:
# Dados de códigos de categoria de mercadorias
df_mcc = spark.read.csv(caminho + 'mcc_codes.csv', header=True, inferSchema=True)

# Exibir o esquema do dataframe
df_mcc.printSchema()

root
 |-- code: integer (nullable = true)
 |-- description: string (nullable = true)



- train_fraud_labels.json

In [58]:
# Processar train_fraud_labels.json
json_path = caminho + 'train_fraud_labels.json'
csv_path = caminho + 'train_fraud_labels.csv'
list_columns = ['transaction_id', 'is_fraud']
one_column = 'target'
process_json_to_csv(json_path, csv_path, list_columns, one_column)

Dados extraídos da chave 'target'.
Arquivo CSV gerado com sucesso em dados/train_fraud_labels.csv.


In [59]:
# Dados de rótulos de fraude
df_train_fraud = spark.read.csv(caminho + 'train_fraud_labels.csv', header=True, inferSchema=True)

# Exibir o esquema do dataframe
df_train_fraud.printSchema()

root
 |-- transaction_id: integer (nullable = true)
 |-- is_fraud: string (nullable = true)



## 4. Análise dos dados

- Volumetria dos dataframes

In [60]:
def dataframe_volumetry(df, df_name):
    '''
    Exibe a quantidade de linhas e colunas de um dataframe.
    
    :param df: DataFrame
        O dataframe a ser analisado.
    :param df_name: string
        Nome do dataframe a ser analisado.
    '''
    print(f'Volumetria do \033[3m{df_name}\033[0m')
    print(f'Linhas: {df.count()}')
    print(f'Colunas: {len(df.columns)}\n')

In [61]:
dataframe_volumetry(df_cards, 'df_cards')
dataframe_volumetry(df_transactions, 'df_transactions')
dataframe_volumetry(df_clients, 'df_clients')
dataframe_volumetry(df_mcc, 'df_mcc')
dataframe_volumetry(df_train_fraud, 'df_train_fraud')

Volumetria do df_cards
Linhas: 6146
Colunas: 13

Volumetria do df_transactions
Linhas: 13305915
Colunas: 12

Volumetria do df_clients
Linhas: 2000
Colunas: 14

Volumetria do df_mcc
Linhas: 109
Colunas: 2

Volumetria do df_train_fraud
Linhas: 8914963
Colunas: 2



- Criação de Views temporárias para cada dataframe

In [62]:
df_cards.createOrReplaceTempView('tb_cards')
df_clients.createOrReplaceTempView('tb_clients')
df_mcc.createOrReplaceTempView('tb_mcc')
df_train_fraud.createOrReplaceTempView('tb_train_fraud')
df_transactions.createOrReplaceTempView('tb_transactions')

- Visualizando os dados da tb_cards

In [63]:
df_cards.show(10, truncate=False)

+----+---------+----------+---------------+----------------+-------+---+--------+----------------+------------+--------------+---------------------+----------------+
|id  |client_id|card_brand|card_type      |card_number     |expires|cvv|has_chip|num_cards_issued|credit_limit|acct_open_date|year_pin_last_changed|card_on_dark_web|
+----+---------+----------+---------------+----------------+-------+---+--------+----------------+------------+--------------+---------------------+----------------+
|4524|825      |Visa      |Debit          |4344676511950444|12/2022|623|YES     |2               |$24295      |09/2002       |2008                 |No              |
|2731|825      |Visa      |Debit          |4956965974959986|12/2020|393|YES     |2               |$21968      |04/2014       |2014                 |No              |
|3701|825      |Visa      |Debit          |4582313478255491|02/2024|719|YES     |2               |$46414      |07/2003       |2004                 |No              |
|42 

- Visualizando os dados da tb_clients

In [64]:
df_clients.show(10, truncate=False)

+----+-----------+--------------+----------+-----------+------+------------------------+--------+---------+-----------------+-------------+----------+------------+----------------+
|id  |current_age|retirement_age|birth_year|birth_month|gender|address                 |latitude|longitude|per_capita_income|yearly_income|total_debt|credit_score|num_credit_cards|
+----+-----------+--------------+----------+-----------+------+------------------------+--------+---------+-----------------+-------------+----------+------------+----------------+
|825 |53         |66            |1966      |11         |Female|462 Rose Lane           |34.15   |-117.76  |$29278           |$59696       |$127613   |787         |5               |
|1746|53         |68            |1966      |12         |Female|3606 Federal Boulevard  |40.76   |-73.74   |$37891           |$77254       |$191349   |701         |5               |
|1718|81         |67            |1938      |11         |Female|766 Third Drive         |34.02  

- Visualizando os dados da tb_mcc

In [65]:
df_mcc.show(10, truncate=False)

+----+------------------------------------------+
|code|description                               |
+----+------------------------------------------+
|5812|Eating Places and Restaurants             |
|5541|Service Stations                          |
|7996|Amusement Parks, Carnivals, Circuses      |
|5411|Grocery Stores, Supermarkets              |
|4784|Tolls and Bridge Fees                     |
|4900|Utilities - Electric, Gas, Water, Sanitary|
|5942|Book Stores                               |
|5814|Fast Food Restaurants                     |
|4829|Money Transfer                            |
|5311|Department Stores                         |
+----+------------------------------------------+
only showing top 10 rows



- Visualizando os dados da tb_train_fraud

In [66]:
df_train_fraud.show(10, truncate=False)

+--------------+--------+
|transaction_id|is_fraud|
+--------------+--------+
|10649266      |No      |
|23410063      |No      |
|9316588       |No      |
|12478022      |No      |
|9558530       |No      |
|12532830      |No      |
|19526714      |No      |
|9906964       |No      |
|13224888      |No      |
|13749094      |No      |
+--------------+--------+
only showing top 10 rows



- Visualizando os dados da tb_transactions

In [67]:
df_transactions.show(10, truncate=False)

+-------+-------------------+---------+-------+-------+------------------+-----------+-------------+--------------+-------+----+------+
|id     |date               |client_id|card_id|amount |use_chip          |merchant_id|merchant_city|merchant_state|zip    |mcc |errors|
+-------+-------------------+---------+-------+-------+------------------+-----------+-------------+--------------+-------+----+------+
|7475327|2010-01-01 00:01:00|1556     |2972   |$-77.00|Swipe Transaction |59935      |Beulah       |ND            |58523.0|5499|NULL  |
|7475328|2010-01-01 00:02:00|561      |4575   |$14.57 |Swipe Transaction |67570      |Bettendorf   |IA            |52722.0|5311|NULL  |
|7475329|2010-01-01 00:02:00|1129     |102    |$80.00 |Swipe Transaction |27092      |Vista        |CA            |92084.0|4829|NULL  |
|7475331|2010-01-01 00:05:00|430      |2860   |$200.00|Swipe Transaction |27092      |Crown Point  |IN            |46307.0|4829|NULL  |
|7475332|2010-01-01 00:06:00|848      |3915   |$

## 5. Construção da ABT

- Alterando o nome das colunas de identificação dos dataframes

In [68]:
df_cards = df_cards.withColumnRenamed('id', 'id_card')
df_cards = df_cards.withColumnRenamed('client_id', 'client_id_card')
df_clients = df_clients.withColumnRenamed('id', 'id_client')

- Executando o _join_ entre os dataframes

In [69]:
abt_01 = df_transactions \
    .join(df_cards, df_transactions['card_id'] == df_cards['id_card'], how='inner') \
    .join(df_clients, df_transactions['client_id'] == df_clients['id_client'], how='inner') \
    .join(df_mcc, df_transactions['mcc'] == df_mcc['code'], how='inner') \
    .join(df_train_fraud, df_transactions['id'] == df_train_fraud['transaction_id'], how='left')      

In [70]:
abt_01.show(20, truncate=False)

+--------+-------------------+---------+-------+-------+------------------+-----------+----------------+--------------+-------+----+-------+-------+--------------+----------+---------+----------------+-------+---+--------+----------------+------------+--------------+---------------------+----------------+---------+-----------+--------------+----------+-----------+------+--------------------------------+--------+---------+-----------------+-------------+----------+------------+----------------+----+----------------------------------+--------------+--------+
|id      |date               |client_id|card_id|amount |use_chip          |merchant_id|merchant_city   |merchant_state|zip    |mcc |errors |id_card|client_id_card|card_brand|card_type|card_number     |expires|cvv|has_chip|num_cards_issued|credit_limit|acct_open_date|year_pin_last_changed|card_on_dark_web|id_client|current_age|retirement_age|birth_year|birth_month|gender|address                         |latitude|longitude|per_capita_i

- Criação de View temporária para abt

In [71]:
abt_01.createOrReplaceTempView('tb_abt_01')

In [72]:
dataframe_volumetry(abt_01, 'tb_abt_01')

Volumetria do tb_abt_01
Linhas: 13305915
Colunas: 43



### 5.1. Análise de dados da ABT

- Panorama geral dos dados

In [73]:
query_01 = spark.sql('''
    SELECT
        YEAR(TO_DATE(date, 'yyyy-MM-dd')) AS ANO,
        COUNT(id) AS QTDE_TRANSACOES,
        COUNT(DISTINCT client_id) AS QTDE_CLIENTES,
        SUM(CAST(REPLACE(amount, '$', '') AS DECIMAL(10,2))) AS TOTAL_TRANSACOES,
        COUNT(DISTINCT card_id) AS QTDE_CARTOES,
        COUNT(DISTINCT merchant_id) AS QTDE_COMERCIANTES,
        COUNT(CASE WHEN is_fraud = 'Yes' THEN id END) AS QTDE_FRAUDES,
        ROUND(100 * COUNT(CASE WHEN is_fraud = 'Yes' THEN id END) / COUNT(*), 3) AS TAXA_FRAUDE
    FROM
        tb_abt_01
    GROUP BY
        ANO
    ORDER BY
        ANO;
''').show(20, truncate=False)

+----+---------------+-------------+----------------+------------+-----------------+------------+-----------+
|ANO |QTDE_TRANSACOES|QTDE_CLIENTES|TOTAL_TRANSACOES|QTDE_CARTOES|QTDE_COMERCIANTES|QTDE_FRAUDES|TAXA_FRAUDE|
+----+---------------+-------------+----------------+------------+-----------------+------------+-----------+
|2010|1240880        |1137         |54232556.12     |2896        |27901            |2573        |0.207      |
|2011|1290770        |1167         |55778904.96     |3137        |28153            |37          |0.003      |
|2012|1321672        |1177         |56832410.86     |3245        |28710            |923         |0.07       |
|2013|1352808        |1190         |58284939.62     |3339        |29280            |1337        |0.099      |
|2014|1365537        |1195         |58617820.51     |3418        |28987            |664         |0.049      |
|2015|1388065        |1204         |59514007.43     |3473        |29616            |2189        |0.158      |
|2016|1392

- Análise das transações por cartão

In [74]:
query_02 = spark.sql('''
    SELECT 
        has_chip AS CONTEM_CHIP,
        use_chip AS METODO_AUTENTICACAO, 
        COUNT(*) AS QTDE_TRANSACOES,
        COUNT(CASE WHEN is_fraud = 'Yes' THEN id END) AS QTDE_FRAUDES,
        ROUND(100 * COUNT(CASE WHEN is_fraud = 'Yes' THEN id END) / COUNT(*), 3) AS TAXA_FRAUDE
    FROM 
        tb_abt_01
    GROUP BY 
        has_chip, use_chip
    ORDER BY 
        QTDE_TRANSACOES DESC;
''').show(20, truncate=False)

+-----------+-------------------+---------------+------------+-----------+
|CONTEM_CHIP|METODO_AUTENTICACAO|QTDE_TRANSACOES|QTDE_FRAUDES|TAXA_FRAUDE|
+-----------+-------------------+---------------+------------+-----------+
|YES        |Swipe Transaction  |5774805        |801         |0.014      |
|YES        |Chip Transaction   |4780818        |3176        |0.066      |
|YES        |Online Transaction |1419172        |8155        |0.575      |
|NO         |Swipe Transaction  |1192380        |576         |0.048      |
|NO         |Online Transaction |138740         |624         |0.45       |
+-----------+-------------------+---------------+------------+-----------+



- Transações por faixa de valor

In [75]:
query_03 = spark.sql('''
    SELECT
        CASE
            WHEN CAST(REPLACE(amount, '$', '') AS DECIMAL(10,2)) BETWEEN 0 AND 50 THEN '0-50'
            WHEN CAST(REPLACE(amount, '$', '') AS DECIMAL(10,2)) BETWEEN 51 AND 100 THEN '51-100'
            WHEN CAST(REPLACE(amount, '$', '') AS DECIMAL(10,2)) BETWEEN 101 AND 500 THEN '101-500'
            WHEN CAST(REPLACE(amount, '$', '') AS DECIMAL(10,2)) BETWEEN 501 AND 1000 THEN '501-1000'
            ELSE '1000+'
        END AS value_range,
        COUNT(*) AS QTDE_TRANSACOES,
        COUNT(CASE WHEN is_fraud = 'Yes' THEN id END) AS QTDE_FRAUDES,
        ROUND(100 * COUNT(CASE WHEN is_fraud = 'Yes' THEN id END) / COUNT(*), 3) AS TAXA_FRAUDE
    FROM 
        tb_abt_01
    GROUP BY 
        value_range
    ORDER BY 
        QTDE_TRANSACOES DESC;                  
''').show(20, truncate=False)  

+-----------+---------------+------------+-----------+
|value_range|QTDE_TRANSACOES|QTDE_FRAUDES|TAXA_FRAUDE|
+-----------+---------------+------------+-----------+
|0-50       |8198871        |4943        |0.06       |
|51-100     |2939552        |2761        |0.094      |
|101-500    |1366430        |4690        |0.343      |
|1000+      |767187         |687         |0.09       |
|501-1000   |33875          |251         |0.741      |
+-----------+---------------+------------+-----------+



- Análise das transações e valores fraudulentos

In [76]:
query_04 = spark.sql('''
    SELECT 
        YEAR(TO_DATE(date, 'yyyy-MM-dd')) AS ANO,
        COUNT(*) AS TOTAL_TRANSACOES,
        COUNT(CASE WHEN is_fraud = 'Yes' THEN id END) AS QTDE_FRAUDES,
        ROUND(100 * COUNT(CASE WHEN is_fraud = 'Yes' THEN id END) / COUNT(*), 2) AS TAXA_FRAUDE,
        AVG(CASE WHEN is_fraud = 'Yes' AND CAST(REPLACE(amount, '$', '') AS DECIMAL(10,2)) >= 0 
                THEN CAST(REPLACE(amount, '$', '') AS DECIMAL(10,2)) 
            END) AS MEDIA_VALORES_FRAUDES,
        SUM(CASE WHEN is_fraud = 'Yes' AND CAST(REPLACE(amount, '$', '') AS DECIMAL(10,2)) >= 0 
                THEN CAST(REPLACE(amount, '$', '') AS DECIMAL(10,2)) 
            END) AS TOTAL_VALORES_FRAUDES,
        MAX(CASE WHEN is_fraud = 'Yes' THEN CAST(REPLACE(amount, '$', '') AS DECIMAL(10,2)) 
            END) AS MAIOR_VALOR_FRAUDADO,
        MIN(CASE WHEN is_fraud = 'Yes' AND CAST(REPLACE(amount, '$', '') AS DECIMAL(10,2)) >= 0 
                THEN CAST(REPLACE(amount, '$', '') AS DECIMAL(10,2)) 
            END) AS MENOR_VALOR_FRAUDADO
    FROM
        tb_abt_01
    GROUP BY
        ANO
    ORDER BY
        ANO;
''').show(20, truncate=False)

+----+----------------+------------+-----------+---------------------+---------------------+--------------------+--------------------+
|ANO |TOTAL_TRANSACOES|QTDE_FRAUDES|TAXA_FRAUDE|MEDIA_VALORES_FRAUDES|TOTAL_VALORES_FRAUDES|MAIOR_VALOR_FRAUDADO|MENOR_VALOR_FRAUDADO|
+----+----------------+------------+-----------+---------------------+---------------------+--------------------+--------------------+
|2010|1240880         |2573        |0.21       |141.412778           |345612.83            |4978.45             |0.00                |
|2011|1290770         |37          |0.0        |170.364706           |5792.40              |540.30              |0.22                |
|2012|1321672         |923         |0.07       |90.272121            |80432.46             |1073.75             |0.01                |
|2013|1352808         |1337        |0.1        |135.406258           |172236.76            |2263.61             |0.01                |
|2014|1365537         |664         |0.05       |129.520

- Análise da distribuição das transações por faixa etária

In [77]:
query_05 = spark.sql('''
    SELECT
        CASE
            WHEN current_age BETWEEN 0 AND 10 THEN '0-10'
            WHEN current_age BETWEEN 11 AND 20 THEN '11-20'
            WHEN current_age BETWEEN 21 AND 30 THEN '21-30'
            WHEN current_age BETWEEN 31 AND 40 THEN '31-40'
            WHEN current_age BETWEEN 41 AND 50 THEN '41-50'
            WHEN current_age BETWEEN 51 AND 60 THEN '51-60'
            WHEN current_age BETWEEN 61 AND 70 THEN '61-70'
            WHEN current_age BETWEEN 71 AND 80 THEN '71-80'
            WHEN current_age BETWEEN 81 AND 90 THEN '81-90'
            ELSE '90+'
        END AS FAIXA_ETARIA,
        COUNT(*) AS QTDE_TRANSACOES,
        COUNT(CASE WHEN is_fraud = 'Yes' THEN id END) AS QTDE_FRAUDES,
        MIN(current_age) AS min_age
    FROM tb_abt_01
    GROUP BY FAIXA_ETARIA
    ORDER BY min_age;
''').show(20, truncate=False)

+------------+---------------+------------+-------+
|FAIXA_ETARIA|QTDE_TRANSACOES|QTDE_FRAUDES|min_age|
+------------+---------------+------------+-------+
|21-30       |428671         |318         |23     |
|31-40       |2377562        |2041        |31     |
|41-50       |3396784        |3234        |41     |
|51-60       |2943032        |2880        |51     |
|61-70       |2105384        |2479        |61     |
|71-80       |970658         |1136        |71     |
|81-90       |864655         |1077        |81     |
|90+         |219169         |167         |91     |
+------------+---------------+------------+-------+



- Transações por estado do estabelecimento comercial

In [78]:
query_06 = spark.sql('''
    SELECT 
        COALESCE(merchant_state, 'ONLINE') AS UF_COMERCIANTE,
        COUNT(*) AS QTDE_TRANSACOES,
        COUNT(CASE WHEN is_fraud = 'Yes' THEN id END) AS QTDE_FRAUDES,
        ROUND(100 * COUNT(CASE WHEN is_fraud != 'No' THEN 1 END) / COUNT(*), 3) AS TAXA_FRAUDE
    FROM
        tb_abt_01
    GROUP BY 
        UF_COMERCIANTE
    ORDER BY 
        QTDE_TRANSACOES DESC;
''').show(20, truncate=False) 

+--------------+---------------+------------+-----------+
|UF_COMERCIANTE|QTDE_TRANSACOES|QTDE_FRAUDES|TAXA_FRAUDE|
+--------------+---------------+------------+-----------+
|ONLINE        |1563700        |8779        |0.561      |
|CA            |1427087        |127         |0.009      |
|TX            |1010207        |76          |0.008      |
|NY            |857510         |58          |0.007      |
|FL            |701623         |63          |0.009      |
|OH            |484122         |316         |0.065      |
|IL            |467931         |36          |0.008      |
|NC            |429427         |44          |0.01       |
|PA            |417766         |33          |0.008      |
|MI            |397970         |39          |0.01       |
|GA            |368206         |22          |0.006      |
|NJ            |322227         |42          |0.013      |
|IN            |312470         |26          |0.008      |
|WA            |286525         |23          |0.008      |
|TN           

- Análise de transações por gênero

In [79]:
query_07 = spark.sql(''' 
    SELECT 
        COALESCE(gender, 'Desconhecido') AS GENERO,
        COUNT(*) AS QTDE_TRANSACOES,
        COUNT(CASE WHEN is_fraud = 'Yes' THEN id END) AS QTDE_FRAUDES,
        ROUND(100 * COUNT(CASE WHEN is_fraud = 'Yes' THEN 1 END) / COUNT(*), 3) AS TAXA_FRAUDE
    FROM 
        tb_abt_01
    GROUP BY 
        GENERO
    ORDER BY 
        QTDE_TRANSACOES DESC;
''').show(20, truncate=False) 

+------+---------------+------------+-----------+
|GENERO|QTDE_TRANSACOES|QTDE_FRAUDES|TAXA_FRAUDE|
+------+---------------+------------+-----------+
|Female|6815916        |6982        |0.102      |
|Male  |6489999        |6350        |0.098      |
+------+---------------+------------+-----------+



- Análise de transações por faixa etária e gênero

In [80]:
query_08 = spark.sql('''
    SELECT 
        FAIXA_ETARIA,
        gender,
        COUNT(*) AS QTDE_TRANSACOES,
        COUNT(CASE WHEN is_fraud = 'Yes' THEN id END) AS QTDE_FRAUDES,
        ROUND(100 * COUNT(CASE WHEN is_fraud = 'Yes' THEN id END) / COUNT(*), 3) AS TAXA_FRAUDE
    FROM 
        (SELECT
            CASE
                WHEN current_age BETWEEN 0 AND 10 THEN '0-10'
                WHEN current_age BETWEEN 11 AND 20 THEN '11-20'
                WHEN current_age BETWEEN 21 AND 30 THEN '21-30'
                WHEN current_age BETWEEN 31 AND 40 THEN '31-40'
                WHEN current_age BETWEEN 41 AND 50 THEN '41-50'
                WHEN current_age BETWEEN 51 AND 60 THEN '51-60'
                WHEN current_age BETWEEN 61 AND 70 THEN '61-70'
                WHEN current_age BETWEEN 71 AND 80 THEN '71-80'
                WHEN current_age BETWEEN 81 AND 90 THEN '81-90'
                ELSE '90+'
            END AS FAIXA_ETARIA,
            gender,
            id,
            is_fraud
        FROM tb_abt_01) AS DADOS_FAIXA_ETARIA
    GROUP BY 
        FAIXA_ETARIA, gender
    ORDER BY 
        QTDE_TRANSACOES DESC;             
''').show(20, truncate=False) 

+------------+------+---------------+------------+-----------+
|FAIXA_ETARIA|gender|QTDE_TRANSACOES|QTDE_FRAUDES|TAXA_FRAUDE|
+------------+------+---------------+------------+-----------+
|41-50       |Female|1764919        |1597        |0.09       |
|41-50       |Male  |1631865        |1637        |0.1        |
|51-60       |Male  |1512801        |1432        |0.095      |
|51-60       |Female|1430231        |1448        |0.101      |
|31-40       |Male  |1243654        |1008        |0.081      |
|31-40       |Female|1133908        |1033        |0.091      |
|61-70       |Female|1062819        |1332        |0.125      |
|61-70       |Male  |1042565        |1147        |0.11       |
|81-90       |Female|558634         |715         |0.128      |
|71-80       |Male  |487802         |519         |0.106      |
|71-80       |Female|482856         |617         |0.128      |
|81-90       |Male  |306021         |362         |0.118      |
|21-30       |Female|256586         |159         |0.062

### 5.2. Análise do target da ABT

- Divisão dos dados pelo target

In [81]:
query_09 = spark.sql('''
    SELECT
        is_fraud,
        COUNT(*) AS QTDE_TRANSACOES,
        ROUND(100 * COUNT(*) / (SELECT COUNT(*) FROM tb_abt_01), 3) AS PERCENTUAL
    FROM
        tb_abt_01
    GROUP BY
        is_fraud
    ORDER BY
        QTDE_TRANSACOES DESC;
''').show(20, truncate=False)

+--------+---------------+----------+
|is_fraud|QTDE_TRANSACOES|PERCENTUAL|
+--------+---------------+----------+
|No      |8901631        |66.9      |
|NULL    |4390952        |33.0      |
|Yes     |13332          |0.1       |
+--------+---------------+----------+



- Distribuição do target por ano

In [82]:
query_10 = spark.sql('''
SELECT
    YEAR(TO_DATE(date, 'yyyy-MM-dd')) AS ANO,
    is_fraud,
    COUNT(*) AS QTDE_TRANSACOES,
    ROUND(100 * COUNT(*) / (
        SELECT 
            COUNT(*) 
        FROM 
            tb_abt_01 
        WHERE 
            YEAR(TO_DATE(date, 'yyyy-MM-dd')) = YEAR(TO_DATE(date, 'yyyy-MM-dd'))
        ), 2) AS PERCENTUAL
FROM
    tb_abt_01
GROUP BY
    ANO, is_fraud
ORDER BY
    ANO, is_fraud DESC;               
''').show(50, truncate=False)

+----+--------+---------------+----------+
|ANO |is_fraud|QTDE_TRANSACOES|PERCENTUAL|
+----+--------+---------------+----------+
|2010|Yes     |2573           |0.02      |
|2010|No      |828956         |6.23      |
|2010|NULL    |409351         |3.08      |
|2011|Yes     |37             |0.0       |
|2011|No      |863391         |6.49      |
|2011|NULL    |427342         |3.21      |
|2012|Yes     |923            |0.01      |
|2012|No      |884498         |6.65      |
|2012|NULL    |436251         |3.28      |
|2013|Yes     |1337           |0.01      |
|2013|No      |905967         |6.81      |
|2013|NULL    |445504         |3.35      |
|2014|Yes     |664            |0.0       |
|2014|No      |914409         |6.87      |
|2014|NULL    |450464         |3.39      |
|2015|Yes     |2189           |0.02      |
|2015|No      |928035         |6.97      |
|2015|NULL    |457841         |3.44      |
|2016|Yes     |2448           |0.02      |
|2016|No      |930314         |6.99      |
|2016|NULL 

### 5.3. Gerando uma amostra representativa da ABT

In [83]:
# Amostra 50% dos dados
abt_02 = abt_01.sample(withReplacement=False, fraction=0.50, seed=42)

# Cache o DataFrame para garantir consistência entre as ações
abt_02.cache()

DataFrame[id: int, date: timestamp, client_id: int, card_id: int, amount: string, use_chip: string, merchant_id: int, merchant_city: string, merchant_state: string, zip: double, mcc: int, errors: string, id_card: int, client_id_card: int, card_brand: string, card_type: string, card_number: bigint, expires: string, cvv: int, has_chip: string, num_cards_issued: int, credit_limit: string, acct_open_date: string, year_pin_last_changed: int, card_on_dark_web: string, id_client: int, current_age: int, retirement_age: int, birth_year: int, birth_month: int, gender: string, address: string, latitude: double, longitude: double, per_capita_income: string, yearly_income: string, total_debt: string, credit_score: int, num_credit_cards: int, code: int, description: string, transaction_id: int, is_fraud: string]

In [84]:
abt_02.show(20, truncate=False)

+-------+-------------------+---------+-------+--------+------------------+-----------+----------------+--------------+-------+----+------+-------+--------------+----------+---------------+----------------+-------+---+--------+----------------+------------+--------------+---------------------+----------------+---------+-----------+--------------+----------+-----------+------+-----------------------------+--------+---------+-----------------+-------------+----------+------------+----------------+----+------------------------------------------+--------------+--------+
|id     |date               |client_id|card_id|amount  |use_chip          |merchant_id|merchant_city   |merchant_state|zip    |mcc |errors|id_card|client_id_card|card_brand|card_type      |card_number     |expires|cvv|has_chip|num_cards_issued|credit_limit|acct_open_date|year_pin_last_changed|card_on_dark_web|id_client|current_age|retirement_age|birth_year|birth_month|gender|address                      |latitude|longitude|

In [85]:
# Cria a view temporária para a abt
abt_02.createOrReplaceTempView('tb_abt_02')

In [86]:
query_09 = spark.sql('''
    SELECT
        is_fraud,
        COUNT(*) AS QTDE_TRANSACOES,
        ROUND(100 * COUNT(*) / (SELECT COUNT(*) FROM tb_abt_02), 3) AS PERCENTUAL
    FROM
        tb_abt_02
    GROUP BY
        is_fraud
    ORDER BY
        QTDE_TRANSACOES DESC;
''').show(20, truncate=False)

+--------+---------------+----------+
|is_fraud|QTDE_TRANSACOES|PERCENTUAL|
+--------+---------------+----------+
|No      |4450946        |66.913    |
|NULL    |2194300        |32.988    |
|Yes     |6640           |0.1       |
+--------+---------------+----------+



## 6. Salvando a ABT em formato parquet

In [87]:
dataframe_volumetry(abt_02, 'tb_abt_02')

Volumetria do tb_abt_02
Linhas: 6651886
Colunas: 43



In [88]:
# Diretório onde os dados serão salvos
caminho = f'dados/ABT/'

# Verifica se o diretório ABT já existe
if os.path.exists(caminho):
    # Exportar para Parquet
    abt_02.write.option('compression', 'snappy').parquet(caminho, mode='overwrite')
    # Valida a quantidade de linhas lidas do Parquet
    read_abt_02 = spark.read.parquet(caminho)
    print(f'\nA ABT parquet tem {read_abt_02.count()} linhas.')
else:
    print(f'\nOcorreu um erro: o diretório "{caminho}" não existe!')


A ABT parquet tem 6651886 linhas.
